# Environment Setting

In [27]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dateutil import parser
import time
import datetime
import random
import math
import os

# Restaurant List

In [2]:
def get_availability(tag):
    if tag:
        checkmark = tag.find('span', {'class': 'icon--16-checkmark-v2'})
        closemark = tag.find('span', {'class': 'icon--16-close-v2'})
        if checkmark:
            return "true"
        elif closemark:
            return "false"
    return "n/a"

## First Page Trial

In [24]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the Yelp page to scrape
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York%2C+NY%2C+United+States&sortby=review_count"
city = 'New York'
# Send a GET request to the URL
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    info = pd.DataFrame(columns=['Name','Link', 'Label', 'Score', 'Review Amount','Outdoor seating Available','Delivery Available','Takeout Available','Price Range'])
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the container that holds the restaurant information
    restaurants = soup.find_all('div', class_='y-css-1he6azc')
    restaurant_data = []
    # Loop through each restaurant and extract the information
    for restaurant in restaurants:
      info_tag = restaurant.find('div', class_= 'y-css-cxcdjj')

      # Extract the restaurant's name
      name_tag = info_tag.find('a', class_='y-css-12ly5yx')
      name = name_tag.get_text() if name_tag else 'N/A'
      link = info_tag.find('a')['href']

      # Extract the restaurant's ranking (if available)
      # ranking_tag = info_tag.find('h3', class_='y-css-hcgwj4')
      # ranking_text = ranking_tag.get_text() if ranking_tag else 'N/A'
      # ranking_match = re.search(r'\d+', ranking_text)
      # ranking = ranking_match.group() if ranking_match else 'N/A'

      # Extract the labels/tags (e.g., cuisine type)
      labels_tags = restaurant.find_all('span', class_='y-css-1cn4gbs')
      labels = ', '.join([label.get_text() for label in labels_tags]) if labels_tags else 'N/A'

      reputation_section = restaurant.find('div', class_='y-css-ohs7lg')
      score_tag = restaurant.find('span', class_='y-css-jf9frv')
      score = score_tag.get_text() if score_tag else 'N/A'
      review_tag = restaurant.find_all('span',class_='y-css-wfbtsu')[0]
      review_amt = review_tag.get_text() if review_tag else 'N/A'


      price_range_tag = restaurant.find('span',class_='y-css-mfup3c')
      price_range = price_range_tag.get_text() if price_range_tag else 'N/A'

      location_tag = restaurant.find('div',class_='y-css-1lvo3zq')
      location_name = location_tag.find('span',class_='y-css-wfbtsu')
      location = location_name.get_text() if location_name else 'N/A'

      # Check for outdoor seating availability
      outdoor_seating = get_availability(restaurant.find('div', {'aria-label': ['Outdoor seating Available', 'Outdoor seating Not Available']}))
      delivery_available = get_availability(restaurant.find('div', {'aria-label': ['Delivery Available', 'Delivery Not Available']}))
      takeout_available = get_availability(restaurant.find('div', {'aria-label': ['Takeout Available', 'Takeout Not Available']}))

      restaurant_data.append({
        'Name': name,
        'Link': link,
        #'Ranking': ranking,
        'Location': location,
        'Label': labels,
        'Score': score,
        'Review Amount': review_amt,
        'Outdoor seating Available': outdoor_seating,
        'Delivery Available': delivery_available,
        'Takeout Available': takeout_available,
        'Price Range': price_range
        })
    df = pd.DataFrame(restaurant_data)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Failed to retrieve the page. Status code: 403


## City List: need change to a loop for restaurant list capturing

## Iterations: 24 pages

In [25]:
def restaurant_list_240(citylink):
  # Loop through the pages
  # Initialize the DataFrame
  list_restaurant = pd.DataFrame(columns=['Name','Ranking', 'Labels', 'Score', 'Review', 'Link',
                            #  'Outdoor seating Available', 'Delivery Available', 'Takeout Available'
                             ])
  for i in range(0, 240, 10):
    # URL of the Yelp page to scrape
    url = f"{citylink}&sortby=review_count&start={i}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the container that holds the restaurant information
        restaurants = soup.find_all('div', class_='y-css-1he6azc')
        restaurant_data = []

        # Loop through each restaurant and extract the information
        for restaurant in restaurants:
            info_tag = restaurant.find('div', class_='y-css-1uij873')

            # Extract the restaurant's name
            name_tag = info_tag.find('a', class_='y-css-12ly5yx')
            name = name_tag.get_text() if name_tag else 'N/A'
            link = info_tag.find('a')['href']

            # Extract the restaurant's ranking (if available)
            ranking_tag = info_tag.find('h3', class_='y-css-hcgwj4')
            ranking_text = ranking_tag.get_text() if ranking_tag else 'N/A'
            ranking_match = re.search(r'\d+', ranking_text)
            ranking = ranking_match.group() if ranking_match else 'N/A'

            # Extract the labels/tags (e.g., cuisine type)
            labels_tags = restaurant.find_all('span', class_='y-css-1cn4gbs')
            labels = ', '.join([label.get_text() for label in labels_tags]) if labels_tags else 'N/A'

            #reputation_section = restaurant.find('div', class_='y-css-ohs7lg')
            score_tag = restaurant.find('span', class_='y-css-jf9frv')
            score = score_tag.get_text() if score_tag else 'N/A'
            review_tag = restaurant.find_all('span', class_='y-css-wfbtsu')[0]
            review_amt = review_tag.get_text() if review_tag else 'N/A'

            # Extract availability features
            # outdoor_seating = get_availability(restaurant.find('div', {'aria-label': ['Outdoor seating Available', 'Outdoor seating Not Available']}))
            # delivery_available = get_availability(restaurant.find('div', {'aria-label': ['Delivery Available', 'Delivery Not Available']}))
            # takeout_available = get_availability(restaurant.find('div', {'aria-label': ['Takeout Available', 'Takeout Not Available']}))

            restaurant_data.append({
                'Name': name,
                'Link': link,
                'Ranking': ranking,
                'Labels': labels,
                'Score': score,
                'Review': review_amt
                # 'Outdoor seating Available': outdoor_seating,
                # 'Delivery Available': delivery_available,
                # 'Takeout Available': takeout_available
            })

        # Append the data to the DataFrame
        df_page = pd.DataFrame(restaurant_data)
        list_restaurant = pd.concat([list_restaurant, df_page], ignore_index=True)
        time.sleep(5)

    else:
      print(f"Failed to retrieve the page. Status code: {response.status_code}")
      break
  return list_restaurant

In [26]:
list_city_6 = {
    # 'nyc':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York%2C+NY%2C+United+States',
    # 'LA':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA%2C+United+States',
    # 'Chicago':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C+IL%2C+United+States',
    # 'Dallas':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Dallas%2C+TX%2C+United+States',
    'Huston':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Huston%2C+ID%2C+United+States'
    # 'WashingtonDC':'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Washington%2C+DC%2C+United+States'
    }

In [ ]:
current_date = datetime.datetime.now().strftime("%Y%m%d")
for key,value in list_city_6.items():
  list_restaurant = restaurant_list_240(value)
  csv_file_name = f'{key}_{current_date}.csv'
  list_restaurant.to_csv(csv_file_name, index=True)

## Main Function

In [ ]:
def restaurant_info(Link):
    url = f'https://www.yelp.com{Link}&sort_by=date_desc'
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    if response.status_code == 200:
      restaurant_list = []
      restaurant_tag = soup.find("h1", class_="y-css-olzveb")
      Restaurant = restaurant_tag.get_text() if restaurant_tag else 'N/A'

      location_tag = soup.find('p', class_='y-css-dg8xxd')
      location = location_tag.get_text() if location_tag else 'N/A'

      review_count_tag = soup.find('span', class_="y-css-loq5qn")
      review_count = review_count_tag.get_text() if review_count_tag else 'N/A'

      price_index_tag = soup.find_all('span', class_='y-css-33yfe')
      price_index = price_index_tag[-1].get_text() if price_index_tag else 'N/A'

      page_tag = soup.find('div', class_='y-css-xdax52')
      pages_total = page_tag.find('span', class_='y-css-wfbtsu')
      pages_total_number = pages_total.get_text() if pages_total else 'N/A'
      try:
        pages_total_number = pages_total_number.split('of ')[-1]
        if pages_total_number.isdigit():
          pages_total_number = int(pages_total_number)
        else:
          pages_total_number = 'Nan'
      except ValueError:
        pages_total_number = 'NA'

      # Find the div with the specific class
      # div_address = soup.find('div', class_='y-css-5qya4l')
      div_element = soup.find('div', class_="arrange__09f24__LDfbs gutter-1-5__09f24__vMtpw y-css-1iy1dwt")

      if div_element:
          # Extract the address and additional information
          address = soup.find('address')
          # address_street = []
          # district = []
          if address:
              address_street_tag = address.find('p',class_='y-css-r4s27p')
              address_street = address_street_tag.get_text() if address_street_tag else 'N/A'
              neighborhood_tag = address.find_all('p',class_='y-css-sauewc')
              if neighborhood_tag:
                if len(neighborhood_tag) >= 2:
                  district = neighborhood_tag[0].get_text() if neighborhood_tag else 'N/A'
                  state_zip = neighborhood_tag[1].get_text() if neighborhood_tag else 'N/A'
                else:
                  district = 'N/A'
                  state_zip = neighborhood_tag[0].get_text() if neighborhood_tag else 'N/A'
              else:
                print('no sauewc')
                district = 'N/A'
                zip_tag = address.find('p',class_='y-css-h9c2fl')
                state_zip = zip_tag.get_text() if zip_tag else 'N/A'
          else:
            print('didnt find address class')
          # Extract the other paragraphs with data-font-weight="semibold"

          semibold_paragraphs = div_element.find_all('p', class_='y-css-jf9frv') #, attrs={"data-font-weight": "semibold"}
          if semibold_paragraphs:
            # print(semibold_paragraphs)
            if len(semibold_paragraphs) >= 2:
              street_tag =  semibold_paragraphs[0]
              street_address = street_tag.get_text() if street_tag else 'N/A'
              block_tag = semibold_paragraphs[1]
              block = block_tag.get_text() if block_tag else 'N/A'
            else:
              street_address = 'N/A'
              block_tag = semibold_paragraphs[0]
              block = block_tag.get_text() if block_tag else 'N/A'
          else:
            print('district cannot be found')
            street_address = 'N/A'
            block = 'N/A'
      else:
          print("Cannot find location section.")

      restaurant_info_series = pd.Series({
        'review count': review_count,
        'price index': price_index,
        'pages': pages_total_number,
        'address': address_street,
        'district': district,
        'zip': state_zip,
        'street': street_address,
        'block': block})
    else:
      print(f"Failed to retrieve the page. Status code: {response.status_code}")
    return restaurant_info_series

### Test examples of restaurant info

In [ ]:
Link = '/biz/daikokuya-los-angeles-7?osq=Restaurants'
df1 = restaurant_info(Link)
df1

review count           (9,665 reviews)
price index                        $$ 
pages                              965
address                   327 E 1st St
district                           N/A
zip              Los Angeles, CA 90012
street                             N/A
block           Little Tokyo, Downtown
dtype: object

In [ ]:
Link = '/biz/l-and-b-spumoni-gardens-brooklyn?osq=Restaurants'
df2 = restaurant_info(Link)
df2

,restaurant,review count,price index,address,district,zip,street,block
0,L & B Spumoni Gardens,"(4,740 reviews)",$$,2725 86th St,N/A,"Brooklyn, NY 11223",11th St & 10th St,Gravesend


In [ ]:
Link = '/biz/beauty-and-essex-new-york-city?osq=Restau...'
df3 = restaurant_info(Link)
df3

,restaurant,review count,price index,address,district,zip,street,block
0,Beauty & Essex,"(3,933 reviews)",$$$,146 Essex St,N/A,"New York City, NY 10002",Stanton St & Rivington St,Lower East Side


## Restaurant list

In [ ]:
list_file_name = '945_restaurant_nyc.csv'
restaurant_list = pd.read_csv(list_file_name).drop_duplicates()
restaurant_list[20:40]

,Unnamed: 0,Name,Ranking,Labels,Score,Review,Link,Outdoor seating Available,Delivery Available,Takeout Available
20,20,Nom Wah Tea Parlor,21,Dim Sum,3.6,(2.4k reviews),/biz/nom-wah-tea-parlor-new-york?osq=Restaurants,False,True,True
21,21,Boucherie West Village,22,"French, Cocktail Bars, Steakhouses",4.5,(2.4k reviews),/biz/boucherie-west-village-new-york-3?osq=Res...,True,True,True
22,22,Butcher Bar,23,"Barbeque, Burgers",4.0,(2.3k reviews),/biz/butcher-bar-astoria-2?osq=Restaurants,True,True,True
23,23,Freemans,24,"Breakfast & Brunch, Lounges, American",3.8,(2.2k reviews),/biz/freemans-new-york-3?osq=Restaurants,True,True,True
24,24,Mad For Chicken Flushing,25,"Korean, Chicken Wings, Gastropubs",4.1,(2.2k reviews),/biz/mad-for-chicken-flushing-flushing-2?osq=R...,False,True,True
25,25,Ayza Wine & Chocolate Bar,26,"Wine Bars, Desserts, New American",4.1,(2.1k reviews),/biz/ayza-wine-and-chocolate-bar-new-york-6?os...,NaN,True,True
26,26,Dutch Freds,27,"Cocktail Bars, New American, Breakfast & Brunch",4.5,(2.1k reviews),/biz/dutch-freds-new-york-2?osq=Restaurants,True,NaN,True
27,27,Traif,28,"New American, Tapas/Small Plates",4.4,(2.1k reviews),/biz/traif-brooklyn?osq=Restaurants,True,True,True
28,28,Upstate Craft Beer & Oyster Bar,29,Seafood,4.4,(2.1k reviews),/biz/upstate-craft-beer-and-oyster-bar-new-yor...,True,False,False
29,29,Lucali,30,Pizza,4.2,(2.1k reviews),/biz/lucali-brooklyn-3?osq=Restaurants,True,True,True


## Combine restaurant list

In [ ]:
import pandas as pd
import glob

# List all CSV files
csv_files = glob.glob('*.csv')

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# Drop duplicate rows
unique_df = combined_df.drop_duplicates()

# Save the resulting DataFrame to a new CSV file
unique_df.to_csv(f'{len(unique_df)}_restaurant_LA.csv', index=False)

## Adding restaurant info to df

In [ ]:
columns_to_add = ['review count', 'price index', 'address', 'district', 'zip', 'street', 'block']
for column in columns_to_add:
    unique_df.loc[:, column] = None

In [ ]:
def pages_number(Link):
    url = f'https://www.yelp.com{Link}&sort_by=date_desc'
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    if response.status_code == 200:
      div_element = soup.find('div', class_="y-css-1iy1dwt", id='reviews')
      page_tag = div_element.find('div', class_='y-css-xdax52')
      pages_total = page_tag.find('span', class_='y-css-wfbtsu')
      pages_total_number = pages_total.get_text() if pages_total else 'N/A'
      try:
        pages_total_number = pages_total_number.split('of ')[-1]
        # if pages_total_number.isdigit():
        #   pages_total_number = int(pages_total_number)
        # else:
        #   pages_total_number = 'Nan'
      except ValueError:
        pages_total_number = 'NA'

      restaurant_info_series = pd.Series({
        'pages': pages_total_number})
    else:
      print(f"Failed to retrieve the page. Status code: {response.status_code}")
    return restaurant_info_series

In [ ]:
Link = '/biz/morrison-atwater-village-los-angeles?osq=...'
df4 = pages_number(Link)
df4

pages    800
dtype: object

In [ ]:
for index, row in unique_df.iloc[0:10].iterrows():
  # if row['pages'] == 'N/A':
    link = row["Link"]
    # Get the restaurant information
    res_info = restaurant_info(link)
    # Update the corresponding row in unique_df with the new data using .loc
    unique_df.loc[index, res_info.index] = res_info
    time.sleep(3)

In [ ]:
count = 0
for index, row in unique_df.iterrows():
  if row['pages'] == 'N/A':
    count += 1
print(count)

38


In [ ]:
unique_df

,Name,Labels,Score,Review,Link,Outdoor seating Available,Delivery Available,Takeout Available,review count,price index,address,district,zip,street,block,pages
0,Bottega Louie,"Italian, Desserts, Breakfast & Brunch",4.1,(18.9k reviews),/biz/bottega-louie-los-angeles?osq=Restaurants,NaN,NaN,NaN,"(18,920 reviews)",$$,700 S Grand Ave,N/A,"Los Angeles, CA 90017",N/A,Downtown,1888
1,Porto’s Bakery & Cafe,"Bakeries, Cafes, Custom Cakes",4.5,(12.1k reviews),/biz/portos-bakery-and-cafe-burbank?osq=Restau...,NaN,NaN,NaN,"(12,125 reviews)",$,3614 W Magnolia Blvd,N/A,"Burbank, CA 91505",N/A,N/A,1207
2,Porto’s Bakery & Cafe,"Bakeries, Cafes, Custom Cakes",4.4,(11.3k reviews),/biz/portos-bakery-and-cafe-glendale?osq=Resta...,NaN,NaN,NaN,"(11,285 reviews)",$,315 N Brand Blvd,N/A,"Glendale, CA 91203",N/A,N/A,1125
3,Wurstküche,"Hot Dogs, German, Gastropubs",4.2,(9.1k reviews),/biz/wurstk%C3%BCche-los-angeles-2?osq=Restaur...,NaN,NaN,NaN,"(9,068 reviews)",$$,800 E 3rd St,N/A,"Los Angeles, CA 90013",N/A,"Arts District, Downtown",906
4,Porto’s Bakery & Cafe,"Bakeries, Custom Cakes, Cafes",4.4,(8.6k reviews),/biz/portos-bakery-and-cafe-downey?osq=Restaur...,NaN,NaN,NaN,"(8,661 reviews)",$,8233 Firestone Blvd,N/A,"Downey, CA 90241",N/A,N/A,859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,Marmalade Cafe - El Segundo,"New American, Breakfast & Brunch, Cafes",3.8,(1.9k reviews),/biz/marmalade-cafe-el-segundo-el-segundo-3?os...,True,True,True,"(1,889 reviews)",$$,2014 Park Pl,N/A,"El Segundo, CA 90245",N/A,N/A,189
389,Daisy Mint,"Thai, Asian Fusion",4.0,(1.9k reviews),/biz/daisy-mint-pasadena?osq=Restaurants,True,True,True,"(1,884 reviews)",$$,1218 E Colorado Blvd,N/A,"Pasadena, CA 91106",N/A,N/A,189
390,Mama Hong’s Vietnamese Kitchen - Burbank,Vietnamese,3.8,(1.9k reviews),/biz/mama-hongs-vietnamese-kitchen-burbank-bur...,False,True,True,"(1,854 reviews)",$$,211 N San Fernando Blvd,N/A,"Burbank, CA 91502",N/A,N/A,186
391,CUT Beverly Hills,"Steakhouses, New American",4.0,(1.8k reviews),/biz/cut-beverly-hills-beverly-hills-3?osq=Res...,True,NaN,True,"(1,845 reviews)",$$$$,9500 Wilshire Blvd,N/A,"Beverly Hills, CA 90212",N/A,N/A,184


In [ ]:
unique_df.to_csv('LA_393_restaurant_list_393_completed.csv')

In [ ]:
import math
reviews = nyc_list['review_count'][0]
start_page = math.ceil(reviews / 10) * 10
start_page

180

In [18]:
nyc_list['url'][10]

'https://www.yelp.com/biz/omar-khayyam-brooklyn-4?adjust_creative=H8bIe4kg_wNS6XDQJClZ9g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=H8bIe4kg_wNS6XDQJClZ9g'

# Review Scrape

In [48]:
nyc_list = pd.read_csv('/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/19_city_dataframe/5883_nyc.csv')

In [ ]:
csv_files = glob.glob('/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/10_nyc_restaurant/restaurant_reviews/*.csv')

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)

In [22]:
test = pd.read_csv('/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/10_nyc_restaurant/restaurant_reviews/nyc_list/00_less1000/718-bistro-brooklyn_82.csv')
test[:10]

,Unnamed: 0,Name,Profile Location,Score,Date,Elite 24,Friends,Reviews,Photos,Helpful,Thanks,Love this,Oh no,reserved,pictures,checkin,Comment,Reply Date,Reply Content
0,0,Lana R.,"Staten Island, Staten Island, NY",5,"Mar 20, 2024",no,0,8,0,0,0,0,0,no,NaN,yes,A Home Cook's Humble Admission: 718 Bistro \nA...,NaN,NaN
1,1,Anna R.,"Brooklyn, NY",2,"Jun 22, 2024",no,4,22,16,2,0,0,0,no,NaN,yes,"Well, well, well... Friends recommended this p...",NaN,NaN
2,2,Irene Z.,"Staten Island, NY",4,"Mar 7, 2024",no,227,192,244,0,0,0,0,no,2 photos,yes,Was here for a banquet the other day and I got...,NaN,NaN
3,3,Maya B.,"Orlando, FL",5,"Jul 3, 2024",no,141,298,805,1,0,1,0,no,1 photo,yes,We have been here for occasions on and off sin...,"Feb 7, 2019",Thank you for your kind words Maya. Our chef a...
4,4,Rob M.,"Huntington, NY",1,"Jun 11, 2024",no,239,7,0,0,0,1,1,no,NaN,yes,Came into dine this past Sunday evening and to...,NaN,NaN
5,5,Leon G.,"Saratoga Springs, NY",5,"Feb 23, 2024",no,14,2,0,0,0,0,0,no,NaN,yes,"Great tasty food, big dishes and great service...",NaN,NaN
6,6,Dmitriy K.,"Brooklyn, Brooklyn, NY",5,"Jun 19, 2024",no,0,7,0,0,0,0,0,no,NaN,yes,I tried crepes with meat and dumplings with ch...,NaN,NaN
7,7,Lenny B.,"Brooklyn, NY",5,"Aug 21, 2023",no,6,82,0,6,0,6,0,no,NaN,yes,"Excellent place, bar, food, service. \nThe wai...",NaN,NaN
8,8,Jackie S.,"Oceanside, NY",4,"Mar 27, 2022",no,1281,1325,5594,3,0,4,0,no,4 photos,yes,I really enjoyed my experience at 718 Bistro. ...,NaN,NaN
9,9,Dmitry D.,"Queens, NY",5,"May 18, 2022",no,654,599,1836,0,0,0,0,no,6 photos,yes,Really good service \nFlavors we're wow on poi...,NaN,NaN


In [24]:
nyc_list[nyc_list['alias'] == '718-bistro-brooklyn']

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes,neighborhood
5,5,L3TbsXPp8vWh5H4IE9-DHw,718-bistro-brooklyn,718 Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/9363Xc...,False,https://www.yelp.com/biz/718-bistro-brooklyn?a...,82,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.1,"{'latitude': 40.580752657504, 'longitude': -73...",['delivery'],NaN,"{'address1': '360 Neptune Ave', 'address2': ''...",1.718975e+10,(718) 975-1999,277.410779,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_url': 'https://www.718bistro.com', ...",Brighton Beach


In [44]:
def elite_checkin(Link, page):
    if page == 0:
      url = Link
    else:
      url = f'{Link}&start={page}'
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    if response.status_code == 200:
      # review_section = soup.find('div', class_='y-css-1iy1dwt', id='reviews')
      review_section = soup.find('section', {'aria-label': 'Recommended Reviews'})
      reviews = review_section.find_all('li', class_='y-css-1jp2syp')
      reviews = reviews[:(len(reviews)-1)]
      restaurant_data = []

      for review in reviews:
        elite_tag = review.find('span', class_='y-css-13vld4t')
        elite_24 = "yes" if elite_tag else "no"

        # Features
        features_section = review.find_all('div',class_='y-css-cluvhg')[0]
        if features_section:
          reserved_tag = features_section.find('span', class_='y-css-pw0opj', title="This Yelper made a reservation for this business on Yelp")
          reserved = 'yes' if reserved_tag else 'no'

          checked_in_tag = features_section.find_all('span', class_='y-css-atw6tm')
          check_in = 'N/A'
          for span in checked_in_tag:
            span_text = span.get_text()
            if 'check-in' in span_text or 'check-ins' in span_text:
              check_in = 'yes'
              break 
        else:
          reserved = 'N/A'
          check_in = 'N/A'

        restaurant_data.append({
                'Elite 24': elite_24,
                'reserved': reserved,
                'checkin': check_in,
                })
    else:
      print(f"Failed to retrieve the page. Status code: {response.status_code}")
    return pd.DataFrame(restaurant_data)

In [ ]:
csv_files = glob.glob('/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/10_nyc_restaurant/restaurant_reviews/*.csv')

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    info = pd.read_csv(file)
    name = os.path.basename(file).split('_')[0]
    Link = nyc_list[nyc_list['alias'] == name]['url']
    reviews = nyc_list[nyc_list['alias'] == name]['review_count']
    end_page = math.ceil(reviews/10)*10
    for page in range(0,end_page,10):
        for index, row in test.iloc[:10].iterrows():
            df = elite_checkin(Link, page)
            test.at[index, 'Elite 24'] = df.at[0, 'Elite 24']
            test.at[index, 'reserved'] = df.at[0, 'reserved']
            test.at[index, 'checkin'] = df.at[0, 'checkin']

In [42]:
test[0:10]

,Unnamed: 0,Name,Profile Location,Score,Date,Elite 24,Friends,Reviews,Photos,Helpful,Thanks,Love this,Oh no,reserved,pictures,checkin,Comment,Reply Date,Reply Content
0,0,Lana R.,"Staten Island, Staten Island, NY",5,"Mar 20, 2024",yes,0,8,0,0,0,0,0,no,NaN,N/A,A Home Cook's Humble Admission: 718 Bistro \nA...,NaN,NaN
1,1,Anna R.,"Brooklyn, NY",2,"Jun 22, 2024",yes,4,22,16,2,0,0,0,no,NaN,N/A,"Well, well, well... Friends recommended this p...",NaN,NaN
2,2,Irene Z.,"Staten Island, NY",4,"Mar 7, 2024",yes,227,192,244,0,0,0,0,no,2 photos,N/A,Was here for a banquet the other day and I got...,NaN,NaN
3,3,Maya B.,"Orlando, FL",5,"Jul 3, 2024",yes,141,298,805,1,0,1,0,no,1 photo,N/A,We have been here for occasions on and off sin...,"Feb 7, 2019",Thank you for your kind words Maya. Our chef a...
4,4,Rob M.,"Huntington, NY",1,"Jun 11, 2024",yes,239,7,0,0,0,1,1,no,NaN,N/A,Came into dine this past Sunday evening and to...,NaN,NaN
5,5,Leon G.,"Saratoga Springs, NY",5,"Feb 23, 2024",yes,14,2,0,0,0,0,0,no,NaN,N/A,"Great tasty food, big dishes and great service...",NaN,NaN
6,6,Dmitriy K.,"Brooklyn, Brooklyn, NY",5,"Jun 19, 2024",yes,0,7,0,0,0,0,0,no,NaN,N/A,I tried crepes with meat and dumplings with ch...,NaN,NaN
7,7,Lenny B.,"Brooklyn, NY",5,"Aug 21, 2023",yes,6,82,0,6,0,6,0,no,NaN,N/A,"Excellent place, bar, food, service. \nThe wai...",NaN,NaN
8,8,Jackie S.,"Oceanside, NY",4,"Mar 27, 2022",yes,1281,1325,5594,3,0,4,0,no,4 photos,N/A,I really enjoyed my experience at 718 Bistro. ...,NaN,NaN
9,9,Dmitry D.,"Queens, NY",5,"May 18, 2022",yes,654,599,1836,0,0,0,0,no,6 photos,N/A,Really good service \nFlavors we're wow on poi...,NaN,NaN


In [ ]:
file = '/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/10_nyc_restaurant/restaurant_reviews/nyc_list/00_less1000/718-bistro-brooklyn_82.csv'
info = pd.read_csv(file)
name = os.path.basename(file).split('_')[0]
Link = nyc_list[nyc_list['alias'] == name]['url']
reviews = nyc_list[nyc_list['alias'] == name]['review_count']
# end_page = math.ceil(reviews/10)*10
end_page = 10
for page in range(0,end_page,10):
    for index, row in test.iloc[page:(page+10)].iterrows():
        df = elite_checkin(Link, page)
        test.at[index, 'Elite 24'] = df.at[0, 'Elite 24']
        test.at[index, 'reserved'] = df.at[0, 'reserved']
        test.at[index, 'checkin'] = df.at[0, 'checkin']

In [39]:
Link

5    https://www.yelp.com/biz/718-bistro-brooklyn?a...
Name: url, dtype: object

In [46]:
def scrape_restaurant_review(Link, page):
    if page == 0:
      url = Link
    else:
      url = f'{Link}&start={page}'
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    if response.status_code == 200:
      # review_section = soup.find('div', class_='y-css-1iy1dwt', id='reviews')
      review_section = soup.find('section', {'aria-label': 'Recommended Reviews'})
      reviews = review_section.find_all('li', class_='y-css-1jp2syp')
      reviews = reviews[:(len(reviews)-1)]
      restaurant_data = []

      for review in reviews:
        #author information
        author_tag = review.find('a',class_='y-css-12ly5yx')
        author = author_tag.get_text()if author_tag else 'N/A'

        author_locations = review.find('span',class_="y-css-h9c2fl")
        author_location = author_locations.get_text() if author_locations else 'N/A'

        elite_tag = review.find('span', class_='y-css-13vld4t')
        elite_24 = "yes" if elite_tag else "no"

        friends_tag = review.find('div', {'aria-label': 'Friends'})
        friends = friends_tag.find('span', class_='y-css-evk3sl').get_text() if friends_tag else '0'

        reviews_tag = review.find('div', {'aria-label': 'Reviews'})
        reviews = reviews_tag.find('span', class_='y-css-evk3sl').get_text() if reviews_tag else '0'

        photos_tag = review.find('div', {'aria-label': 'Photos'})
        photos = photos_tag.find('span', class_='y-css-evk3sl').get_text() if photos_tag else '0'

        #overall review
        score_tag = review.find('div', {'aria-label': re.compile(r' star rating')})
        score = re.search(r'\d+', score_tag['aria-label']).group() if score_tag else 'N/A'
        date_tag = review.find('span', class_='y-css-wfbtsu')
        date = date_tag.get_text() if date_tag else 'N/A'

        # Features
        features_section = review.find_all('div',class_='y-css-cluvhg')[0]
        if features_section:
          reserved_tag = features_section.find('span', class_='y-css-pw0opj', title="This Yelper made a reservation for this business on Yelp")
          reserved = 'yes' if reserved_tag else 'no'

          pictures_tag = features_section.find('a', class_='y-css-12ly5yx')
          pictures = pictures_tag.get_text() if pictures_tag else 'N/A'

          # checked_in_tag = features_section.find_all('span', class_='y-css-pw0opj')
          # checked_in = checked_in_tag.get_text() if checked_in_tag else 'N/A'
          # check_in = 'N/A'
          # if checked_in == '1 check-in' or '2 check-ins':
          #   check_in = 'yes'
          checked_in_tag = features_section.find_all('span', class_='y-css-atw6tm')
          check_in = 'N/A'
          for span in checked_in_tag:
            span_text = span.get_text()
            if 'check-in' in span_text or 'check-ins' in span_text:
              check_in = 'yes'
              break
        else:
          reserved = 'N/A'
          pictures = 'N/A'
          check_in = 'N/A'

        #review content
        review_content_tag = review.find('span', class_='raw__09f24__T4Ezm')
        review_content = review_content_tag.get_text(separator='\n') if review_content_tag else 'N/A'

        reaction_section = review.find('div', class_='y-css-o1wrcf')
        reaction_divs = reaction_section.find_all('div',class_='y-css-1b9tgy5')
        helpful = '0'
        thanks = '0'
        love_this = '0'
        oh_no = '0'

        for div in reaction_divs:
          label_tag = div.find('span', class_='y-css-16avcyn')
          count_tag = div.find('span', class_='y-css-q3ifzf')
          if label_tag and count_tag:
              if 'Helpful' in label_tag.get_text():
                helpful = count_tag.get_text()
              elif 'Thanks' in label_tag.get_text():
                thanks = count_tag.get_text()
              elif 'Love this' in label_tag.get_text():
                love_this = count_tag.get_text()
              elif 'Oh no' in label_tag.get_text():
                oh_no = count_tag.get_text()
              else:
                '0'
          else:
            print('no tags found')

        # review reply
        reply_section = review.find('div', class_='y-css-1wjx5j4')
        if reply_section:
          reply_date_tag = reply_section.find('p', class_='y-css-wfbtsu')
          reply_date = reply_date_tag.get_text() if reply_date_tag else 'N/A'
          reply_content_tag = reply_section.find('span', class_="raw__09f24__T4Ezm")
          reply_content = reply_content_tag.get_text(separator='\n') if reply_content_tag else 'N/A'
        else:
          reply_date = 'N/A'
          reply_content = 'N/A'

        restaurant_data.append({
                'Name': author,
                'Profile Location': author_location,
                'Score': score,
                'Date': date,
                'Elite 24': elite_24,
                'Friends': friends,
                'Reviews': reviews,
                'Photos': photos,
                'Helpful': helpful,
                'Thanks': thanks,
                'Love this': love_this,
                'Oh no': oh_no,
                'reserved': reserved,
                'pictures': pictures,
                'checkin': check_in,
                'Comment': review_content,
                'Reply Date': reply_date,
                'Reply Content': reply_content
                })
        # time.sleep(random.randint(1,60))
    else:
      print(f"Failed to retrieve the page. Status code: {response.status_code}")
    return pd.DataFrame(restaurant_data)#, pd.DataFrame(restaurant_list)

### Run the function and concatenate them into a single DataFrame

In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dateutil import parser
import time

In [ ]:
combined_df = pd.read_csv('/Users/juliazhou/Desktop/MADS/Courses/S4/Thesis/backlog/11_LA_restaurant/restaurant_list/LA_393_restaurant_list.csv')
# combined_df

In [ ]:
combined_df[2:3]

,Unnamed: 0,Name,Labels,Score,Review,Link,Outdoor seating Available,Delivery Available,Takeout Available,review count,price index,address,district,zip,street,block,pages
2,2,Porto’s Bakery & Cafe,"Bakeries, Cafes, Custom Cakes",4.4,(11.3k reviews),/biz/portos-bakery-and-cafe-glendale?osq=Resta...,NaN,NaN,NaN,"(11,285 reviews)",$,315 N Brand Blvd,NaN,"Glendale, CA 91203",NaN,NaN,1125


In [ ]:
sample_nyc_100 = combined_df.sample(n=100).reset_index(drop=True)
sample_nyc_100

,Unnamed: 0,Name,Ranking,Labels,Score,Review,Link,Outdoor seating Available,Delivery Available,Takeout Available
0,77,Rolfs,78,"German, Bars",2.3,(1.4k reviews),/biz/rolfs-new-york?osq=Restaurants,NaN,NaN,NaN
1,140,Nick’s Pizza - Forest Hills - Temp. CLOSED,141,Pizza,4.0,(970 reviews),/biz/nicks-pizza-forest-hills-forest-hills-2?o...,True,True,True
2,138,New World Mall Food Court,139,Food Court,4.1,(953 reviews),/biz/new-world-mall-food-court-flushing?osq=Re...,NaN,NaN,NaN
3,169,Clover Club,170,"Lounges, American, Cocktail Bars",4.1,(885 reviews),/biz/clover-club-brooklyn?osq=Restaurants,True,True,True
4,208,Tick Tock Diner,209,Diners,2.6,(754 reviews),/biz/tick-tock-diner-clifton-4?osq=Restaurants,True,True,True
...,...,...,...,...,...,...,...,...,...,...
95,115,Soba Totto,116,"Japanese, Salad, Bars",3.8,(952 reviews),/biz/soba-totto-new-york?osq=Restaurants,False,True,True
96,199,Tsurutontan Udon Noodle Brasserie - Midtown,200,"Japanese, Noodles, Soup",3.7,(764 reviews),/biz/tsurutontan-udon-noodle-brasserie-midtown...,NaN,NaN,NaN
97,78,EN Japanese Brasserie,79,"Japanese, Sushi Bars, Bars",3.8,(1.4k reviews),/biz/en-japanese-brasserie-new-york?osq=Restau...,True,True,True
98,226,Tasty Dumpling,237,Chinese,3.7,(642 reviews),/biz/tasty-dumpling-new-york?osq=Restaurants,False,True,True


In [ ]:
nyc_list['url'][1]

'https://www.yelp.com/biz/tone-cafe-brooklyn-2?adjust_creative=H8bIe4kg_wNS6XDQJClZ9g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=H8bIe4kg_wNS6XDQJClZ9g'

In [ ]:
info = pd.DataFrame(columns=['Name', 'Profile Location', 'Score', 'Date',
                                 'Elite 24', 'Friends', 'Reviews', 'Photos', 'reserved', 'pictures', 'checkin',
                                 'Comment', 'Helpful', 'Thanks', 'Love this', 'Oh no'])

In [ ]:
restaurant_dataframes = {}
# Loop through each restaurant link

for index, row in nyc_list.iloc[6:7].iterrows():
    restaurant_name = row["alias"]
    print(restaurant_name)
    link = row["url"]
    end_page = math.ceil(row['review_count'] / 10) * 10
    print(end_page)
    for page in range(110,end_page,10):
        page_info = scrape_restaurant_review(link, page)
        info = pd.concat([info, page_info], ignore_index=True)
        time.sleep(random.randint(1, 60))
    restaurant_dataframes[restaurant_name] = info
    filename = f"{restaurant_name}.csv"
    info.to_csv(filename, index=False)

In [ ]:
import glob
import os

# List all CSV files
csv_files = glob.glob('*.csv')

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Extract the restaurant name from the file name (without extension)
    restaurant_name = os.path.splitext(os.path.basename(file))[0]

    # Add a column with the restaurant name
    df['restaurant_name'] = restaurant_name

    # Ensure the restaurant name is the first column
    df = df[['restaurant_name'] + [col for col in df.columns if col != 'restaurant_name']]

    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
count = merged_df['restaurant_name'].value_counts()
count

restaurant_name
Red Hook Lobster Pound       1310
Buddakan                      100
The Halal Guys                100
Amy Ruth’s                    100
Ippudo NY                     100
Serendipity 3                 100
LoveMama                      100
Di Fara Pizza                 100
Artichoke Basille’s Pizza     100
L & B Spumoni Gardens         100
Name: count, dtype: int64

In [ ]:
red_hook_lobster_pound_100 = combined_df[combined_df['restaurant_name'] == 'Red Hook Lobster Pound'].sample(n=100, random_state=1)

In [ ]:
red_hook_lobster_pound_100.to_csv('red_hook_lobster_pound_100.csv', index=False)

In [ ]:
reviews_df = pd.read_csv('Red Hook Lobster Pound.csv')